# <span style='font-family:"Times New Roman"'> <span styel=''> **INTERPRETATION**

*Emile Cohen* 
    
*March 2020*

**Goal:** In this Notebook, we gather code allowin to interpret the cancer panels computed in this section.



---

In [2]:
%run -i '../../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')
from cancer_utils import *
from scipy.stats import fisher_exact, ranksums, chi2, norm
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.gridspec as gridspec

data_path = '../../../data/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

In [17]:
cancer_type = 'Breast Cancer'
metrics = 'frac_genome_altered'
master = load_clean_up_master(data_path + 'merged_data/master_file.pkl')
# we remover indeterminate and NA tp53_cn_state samples
master_cancer = master[master['Cancer_Type'] == cancer_type][master['tp53_cn_state'] != 'INDETERMINATE'][~master['tp53_cn_state'].isna()]

group_1 = master_cancer[master_cancer['tp53_res_group'] == 'tp53_res'][metrics]
group_2 = master_cancer[master_cancer['tp53_res_group'] == 'no_tp53_res'][metrics]

median_1 = group_1.median()
median_2 = group_2.median()

statistic, p_value = ranksums(group_1.dropna().values,group_2.dropna().values)

print('median tp53_res: ' + str(median_1))
print('median no_tp53_res: ' + str(median_2))
print('statistic, p-value: ' + str(statistic) +','+ str(p_value))

median tp53_res: 0.229
median no_tp53_res: 0.6495
statistic, p-value: -26.043323653020323,1.6012805178148291e-149


In [5]:
breast_cancer_clinical = pd.read_csv(data_path + 'cbioportal/raw/Breast_Cancer/breast_msk_2018_clinical_data.tsv', sep='\t')

In [6]:
breast_cancer_clinical.head()

,Study ID,Patient ID,Sample ID,Cancer Type,Cancer Type Detailed,Disease Free Event,Disease Free (Months),ER PCT Primary,ER Status of Sequenced Sample,ER Status of the Primary,Fraction Genome Altered,HER2 FISH Status of Sequenced Sample,HER2 FISH Ratio Value of Sequenced Sample,HER2 FISH Ratio Primary,HER2 FISH Status (Report and ASCO) of Primary,HER2 IHC Status Primary,HER2 IHC Score of Sequenced Sample,HER2 IHC Status of Sequenced Sample,HER2 IHC Score Primary,HER2 Primary Status,Overall HR Status of Sequenced Sample,Invasive Carcinoma Diagnosis Age,Time to Diagnosis Invasive Carcinoma,Last Communication Contact,Primary Tumor Laterality,Menopausal Status At Diagnosis,Metastatic Disease at Last Follow-up,Metastatic Recurrence Time,Mutation Count,M Stage,NGS Sample Collection Time Period,N Stage,Oncotree Code,Overall Survival (Months),Overall Survival Status,Overall HER2 Status of Sequenced Sample,Overall Patient HER2 Status,Overall Patient HR Status,Overall Patient Receptor Status,Overall Primary Tumor Grade,Primary Nuclear Grade,Prior Breast Primary,Prior Local Recurrence,PR PCT Primary,PR Status of Sequenced Sample,PR Status of the Primary,Receptor Status Primary,Number of Samples Per Patient,Site of Sample,Sample Type,Sex,Stage At Diagnosis,Time To Death (Months),Tumor Sample Histology,Tumor Tissue Origin,T Stage,Patient's Vital Status
0,breast_msk_2018,P-0000004,P-0000004-T01-IM3,Breast Cancer,Breast Invasive Ductal Carcinoma,1,1.1,70,Positive,Positive,NaN,NaN,NaN,NaN,NaN,Negative,0-1+,Negative,1+,Negative,Positive,37,444.87,14484,Left,Pre,Yes,446.0,4.0,M1,445,N3a,IDC,31.5,LIVING,Negative,Negative,Positive,HR+/HER2-,II Intermediate Grade (Moderately Differentiated),III (High),No,No,22,Positive,Positive,HR+/HER2-,1,Treatment Naive Primary,Primary,Female,IV,NaN,Breast Invasive Ductal Carcinoma,Breast,T1c,Alive
1,breast_msk_2018,P-0000012,P-0000012-T02-IM3,Breast Cancer,Breast Invasive Ductal Carcinoma,0,218.0,0,Negative,Negative,0.6891,NaN,NaN,NaN,NaN,Negative,0-1+,Negative,0-1+,Negative,Negative,43,516.48,22336,Right,Pre,No,NaN,1.0,M0,517,N0,IDC,218.0,LIVING,Negative,Negative,Negative,Triple Negative,III High Grade (Poorly Differentiated),III (High),No,No,0,Negative,Negative,Triple Negative,1,Treatment Naive Primary,Primary,Female,IIA,NaN,Breast Invasive Ductal Carcinoma,Breast,T2,Alive
2,breast_msk_2018,P-0000015,P-0000015-T01-IM3,Breast Cancer,Breast Invasive Ductal Carcinoma,1,68.9,90,Positive,Positive,NaN,NaN,NaN,0.7,Negative,Negative,0+,Negative,0+,Negative,Positive,37,449.80,16656,Right,Pre,Yes,519.0,7.0,M0,534,N1mi,IDC,98.0,DECEASED,Negative,Negative,Positive,HR+/HER2-,III High Grade (Poorly Differentiated),II (Intermediate),No,No,30,Positive,Positive,HR+/HER2-,1,Liver,Metastasis,Female,IB,548.0,Breast Invasive Ductal Carcinoma,Breast,T1b,Deceased
3,breast_msk_2018,P-0000041,P-0000041-T01-IM3,Breast Cancer,Breast Invasive Ductal Carcinoma,1,90.2,50,Positive,Positive,0.6367,Negative,0.24,NaN,NaN,Positive,2+,Equivocal,3+,Positive,Positive,43,513.68,19364,Left,Pre,Yes,604.0,10.0,M0,618,N0,IDC,123.1,DECEASED,Negative,Positive,Positive,HR+/HER2+,III High Grade (Poorly Differentiated),Unknown,No,No,50,Positive,Positive,HR+/HER2+,1,Breast,Metastasis,Female,IA,637.0,Breast Invasive Ductal Carcinoma,Breast,T1b,Deceased
4,breast_msk_2018,P-0000057,P-0000057-T01-IM3,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,1,0.5,80,Positive,Positive,0.5093,NaN,NaN,NaN,NaN,Negative,0-1+,Negative,1+,Negative,Positive,38,458.36,15871,Left,Pre,Yes,459.0,5.0,M1,489,NX,MDLC,63.6,DECEASED,Negative,Negative,Positive,HR+/HER2-,II Intermediate Grade (Moderately Differentiated),II (Intermediate),No,No,50,Positive,Positive,HR+/HER2-,1,Post-Treatment Primary,Primary,Female,IV,522.0,Breast Mixed Ductal and Lobular Carcinoma,Breast,TX,Deceased
